In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statistics as st
import re
from keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import math
import numpy as np

2024-06-02 13:38:57.616292: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-02 13:38:57.789888: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-02 13:38:58.449702: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-02 13:38:59.678274: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
filename = "~/dev/avia/Train.xlsx"
df_economy = pd.read_excel(filename, sheet_name="economy")
df_business = pd.read_excel(filename, sheet_name="business")

In [3]:
df_economy["class"] = [0. for i in range(len(df_economy))]
df_business["class"] = [1. for i in range(len(df_business))]
data = pd.concat([df_economy, df_business])

In [4]:
data

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,class
0,24-03-2022,GO FIRST,G8,7559,15:30,Bangalore,02h 30m,non-stop,18:00,Kolkata,"3,845",0.0
1,08-03-2022,GO FIRST,G8,395,08:10,Bangalore,08h 25m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,16:35,Delhi,"4,028",0.0
2,22-02-2022,Air India,AI,868,18:00,Delhi,16h 55m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,10:55,Chennai,"11,520",0.0
3,23-02-2022,Indigo,6E,438,18:50,Hyderabad,01h 15m,non-stop,20:05,Chennai,"4,453",0.0
4,26-02-2022,Vistara,UK,816,11:30,Bangalore,08h 35m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:05,Mumbai,"7,212",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
74784,24-02-2022,Vistara,UK,951,14:20,Delhi,08h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:50,Bangalore,"42,457",1.0
74785,16-02-2022,Vistara,UK,774,20:35,Kolkata,20h 20m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,16:55,Delhi,"81,519",1.0
74786,23-02-2022,Vistara,UK,876,21:35,Hyderabad,12h 05m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,09:40,Kolkata,"48,851",1.0
74787,20-02-2022,Air India,AI,865,10:40,Delhi,02h 10m,non-stop,12:50,Mumbai,"22,152",1.0


In [5]:
data["from"].unique()

array(['Bangalore', 'Delhi', 'Hyderabad', 'Kolkata', 'Chennai', 'Mumbai'],
      dtype=object)

In [6]:
del data["date"]
del data["num_code"]
del data["ch_code"]

In [7]:
data["airline"].value_counts()

airline
Vistara      102201
Air India     64752
Indigo        34513
GO FIRST      18593
AirAsia       12859
SpiceJet       7206
StarAir          50
Trujet           34
Name: count, dtype: int64

In [8]:
data = data.set_index('airline')
data = data.drop(['StarAir', 'Trujet', 'SpiceJet', 'AirAsia'], axis=0)
data = data.reset_index()
data["airline"].value_counts()

airline
Vistara      102201
Air India     64752
Indigo        34513
GO FIRST      18593
Name: count, dtype: int64

In [9]:
data = data.dropna()
data = data.drop_duplicates()

In [10]:
data

,airline,dep_time,from,time_taken,stop,arr_time,to,price,class
0,GO FIRST,15:30,Bangalore,02h 30m,non-stop,18:00,Kolkata,"3,845",0.0
1,GO FIRST,08:10,Bangalore,08h 25m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,16:35,Delhi,"4,028",0.0
2,Air India,18:00,Delhi,16h 55m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,10:55,Chennai,"11,520",0.0
3,Indigo,18:50,Hyderabad,01h 15m,non-stop,20:05,Chennai,"4,453",0.0
4,Vistara,11:30,Bangalore,08h 35m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:05,Mumbai,"7,212",0.0
...,...,...,...,...,...,...,...,...,...
220030,Vistara,11:30,Mumbai,05h 15m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,16:45,Delhi,"37,670",1.0
220039,Air India,18:00,Delhi,17h 10m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,11:10,Bangalore,"47,714",1.0
220048,Air India,12:10,Delhi,22h 50m,2+-stop,11:00,Chennai,"67,421",1.0
220049,Vistara,15:20,Kolkata,07h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:50,Mumbai,"70,431",1.0


In [11]:
stops = [stop[:10].replace("\t", "").replace("\n", "").replace(" ", "") for stop in data["stop"]]
data["stop"] = stops
data

,airline,dep_time,from,time_taken,stop,arr_time,to,price,class
0,GO FIRST,15:30,Bangalore,02h 30m,non-stop,18:00,Kolkata,"3,845",0.0
1,GO FIRST,08:10,Bangalore,08h 25m,1-stop,16:35,Delhi,"4,028",0.0
2,Air India,18:00,Delhi,16h 55m,1-stop,10:55,Chennai,"11,520",0.0
3,Indigo,18:50,Hyderabad,01h 15m,non-stop,20:05,Chennai,"4,453",0.0
4,Vistara,11:30,Bangalore,08h 35m,1-stop,20:05,Mumbai,"7,212",0.0
...,...,...,...,...,...,...,...,...,...
220030,Vistara,11:30,Mumbai,05h 15m,1-stop,16:45,Delhi,"37,670",1.0
220039,Air India,18:00,Delhi,17h 10m,1-stop,11:10,Bangalore,"47,714",1.0
220048,Air India,12:10,Delhi,22h 50m,2+-stop,11:00,Chennai,"67,421",1.0
220049,Vistara,15:20,Kolkata,07h 30m,1-stop,22:50,Mumbai,"70,431",1.0


In [12]:
data["airline"].unique()

array(['GO FIRST', 'Air India', 'Indigo', 'Vistara'], dtype=object)

In [13]:
data["airline"].unique()

array(['GO FIRST', 'Air India', 'Indigo', 'Vistara'], dtype=object)

In [14]:
def norm(value):
    max_value = max(value)
    min_value = min(value)
    result = [(i-min_value)/(max_value-min_value) for i in value]
    return result

In [15]:
airline_value = list(data["airline"].unique())
airline_array = [airline_value.index(airline) for airline in data["airline"]]
airline_array = norm(airline_array)
data["airline"] = airline_array

In [16]:
data["from"].unique()

array(['Bangalore', 'Delhi', 'Hyderabad', 'Kolkata', 'Chennai', 'Mumbai'],
      dtype=object)

In [17]:
cities = list(data["from"].unique())
from_array = [cities.index(fr) for fr in data["from"]]
to_array = [cities.index(to) for to in data["to"]]
data["from"] = norm(from_array)
data["to"] = norm(to_array)

In [18]:
def time_to_inteval(value):
    interval_value = 6
    minutes = int(value.split(":")[0])
    interval = math.floor(minutes / interval_value)
    return interval

In [19]:
dep_time_array = [time_to_inteval(time) for time in data["dep_time"]]
arr_time_array = [time_to_inteval(time) for time in data["arr_time"]]
data["dep_time"] = norm(dep_time_array)
data["arr_time"] = norm(arr_time_array)

In [20]:
max(arr_time_array)

3

In [21]:
def minutes(value):
    hours_match = re.search(r'(\d+)h', value)
    minutes_match = re.search(r'(\d+)m', value)
    hours = int(hours_match.group(1)) if hours_match else 0
    minutes = int(minutes_match.group(1)) if minutes_match else 0
    return math.floor((hours + minutes/60)/6)

In [22]:
time_taken_array =  [minutes(time) for time in data["time_taken"]]
data["time_taken"] = norm(time_taken_array)

In [23]:
max(time_taken_array)

8

In [24]:
data["time_taken"].value_counts()

time_taken
0.125    20182
0.250    12928
0.000     9444
0.375     6746
0.500     6418
0.625      960
0.750      143
0.875        7
1.000        2
Name: count, dtype: int64

In [25]:
data = data.set_index('time_taken')
data = data.drop([0.625, 0.750, 0.875, 1.000], axis=0)
data = data.reset_index()
data["time_taken"].value_counts()

time_taken
0.125    20182
0.250    12928
0.000     9444
0.375     6746
0.500     6418
Name: count, dtype: int64

In [26]:
data

,time_taken,airline,dep_time,from,stop,arr_time,to,price,class
0,0.000,0.000000,0.666667,0.0,non-stop,1.000000,0.6,"3,845",0.0
1,0.125,0.000000,0.333333,0.0,1-stop,0.666667,0.2,"4,028",0.0
2,0.250,0.333333,1.000000,0.2,1-stop,0.333333,0.8,"11,520",0.0
3,0.000,0.666667,1.000000,0.4,non-stop,1.000000,0.8,"4,453",0.0
4,0.125,1.000000,0.333333,0.0,1-stop,1.000000,1.0,"7,212",0.0
...,...,...,...,...,...,...,...,...,...
55713,0.000,1.000000,0.333333,1.0,1-stop,0.666667,0.2,"37,670",1.0
55714,0.250,0.333333,1.000000,0.2,1-stop,0.333333,0.0,"47,714",1.0
55715,0.375,0.333333,0.666667,0.2,2+-stop,0.333333,0.8,"67,421",1.0
55716,0.125,1.000000,0.666667,0.6,1-stop,1.000000,1.0,"70,431",1.0


In [27]:
data['dep_time'].unique()

array([0.66666667, 0.33333333, 1.        , 0.        ])

In [28]:
stop_values = ['non-stop', '1-stop', '2+-stop']
stop_array = [stop_values.index(stop) for stop in data["stop"]]
data["stop"] = norm(stop_array)

In [29]:
data = data.set_index('dep_time')
data = data.drop([0.0], axis=0)
data = data.reset_index()
data["dep_time"].value_counts()

dep_time
0.333333    23757
0.666667    15825
1.000000    13967
Name: count, dtype: int64

In [30]:
data

,dep_time,time_taken,airline,from,stop,arr_time,to,price,class
0,0.666667,0.000,0.000000,0.0,0.0,1.000000,0.6,"3,845",0.0
1,0.333333,0.125,0.000000,0.0,0.5,0.666667,0.2,"4,028",0.0
2,1.000000,0.250,0.333333,0.2,0.5,0.333333,0.8,"11,520",0.0
3,1.000000,0.000,0.666667,0.4,0.0,1.000000,0.8,"4,453",0.0
4,0.333333,0.125,1.000000,0.0,0.5,1.000000,1.0,"7,212",0.0
...,...,...,...,...,...,...,...,...,...
53544,0.333333,0.000,1.000000,1.0,0.5,0.666667,0.2,"37,670",1.0
53545,1.000000,0.250,0.333333,0.2,0.5,0.333333,0.0,"47,714",1.0
53546,0.666667,0.375,0.333333,0.2,1.0,0.333333,0.8,"67,421",1.0
53547,0.666667,0.125,1.000000,0.6,0.5,1.000000,1.0,"70,431",1.0


In [31]:
data["price"] = [float(price.replace(",", "")) for price in data["price"]]

In [32]:
data["price"].unique()

array([  3845.,   4028.,  11520., ...,  82071., 101531.,  37670.])

In [33]:
corr = {
    "airline": st.correlation(data["airline"], data["price"]),
    "dep_time": st.correlation(data["dep_time"], data["price"]),
    "from": st.correlation(data["from"], data["price"]),
    "time_taken": st.correlation(data["time_taken"], data["price"]),
    "stop": st.correlation(data["stop"], data["price"]),
    "arr_time": st.correlation(data["arr_time"], data["price"]),
    "to": st.correlation(data["to"], data["price"]),
    "class": st.correlation(data["class"], data["price"])
}

In [34]:
corr

{'airline': 0.26394424967119967,
 'dep_time': 0.0539676294251541,
 'from': 0.013622120925250818,
 'time_taken': 0.16754340326239983,
 'stop': 0.06944986338741085,
 'arr_time': 0.005206900281462125,
 'to': 0.0031836507254306696,
 'class': 0.9295202329055136}

In [35]:
data["from"].unique()

array([0. , 0.2, 0.4, 0.6, 0.8, 1. ])

In [36]:
data['airline'].std()

0.3230953320979538

In [37]:
data = data.dropna()
data = data.drop_duplicates()
data = data.sample(frac=1).reset_index(drop=True)

In [38]:
targets = data["price"]
del data["price"]

In [39]:
data

,dep_time,time_taken,airline,from,stop,arr_time,to,class
0,0.666667,0.375,0.333333,0.4,0.5,0.666667,0.2,0.0
1,0.333333,0.250,0.333333,0.0,0.5,1.000000,0.6,0.0
2,0.666667,0.125,0.666667,1.0,0.5,1.000000,0.6,0.0
3,0.333333,0.125,0.666667,1.0,0.5,0.666667,0.8,0.0
4,0.333333,0.250,1.000000,1.0,0.5,1.000000,0.4,0.0
...,...,...,...,...,...,...,...,...
37265,0.333333,0.125,0.666667,0.2,0.5,0.666667,0.6,0.0
37266,0.333333,0.125,0.666667,0.2,0.5,0.666667,0.8,0.0
37267,1.000000,0.125,0.666667,0.6,0.5,0.000000,0.4,0.0
37268,1.000000,0.250,1.000000,0.0,0.5,0.333333,0.6,0.0


In [40]:
data['class'].value_counts()

class
0.0    30470
1.0     6800
Name: count, dtype: int64

In [41]:
data.shape
counts_train = (data.shape[0]/100)*80
counts_train

29816.0

In [98]:
counts_train = round((data.shape[0]/100)*80)
counts_tests = counts_train + round((data.shape[0]/100)*15)

x_train = data[:counts_train]
y_train = targets[:counts_train]

x_test = data[counts_train+1:counts_tests]
y_test = targets[counts_train+1:counts_tests]

x_validate = data[counts_tests+1:]
y_validate = targets[counts_tests+1:]

In [99]:
model = models.Sequential()
model.add(layers.Input(shape=(8, )))
model.add(layers.Dense(4, activation = "relu"))
model.add(layers.Dense(8, activation = "relu"))
model.add(layers.Dense(2, activation = "relu"))
model.add(layers.Dense(1, activation = "relu"))

In [100]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_26 (Dense)                │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 8)              │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 2)              │            18 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97 (388.00 B)

 Trainable params: 97 (388.00 B)

 Non-trainable params: 0 (0.00 B)

In [101]:
model.compile(loss='mean_squared_error', optimizer=Adam(), metrics=['mape'])

In [102]:
results = model.fit(x_train, y_train, epochs= 20, batch_size = 8, validation_data = (x_validate, y_validate))

Epoch 1/20
3727/3727 ━━━━━━━━━━━━━━━━━━━━ 4s 733us/step - loss: 691954048.0000 - mape: 92.2812 - val_loss: 362236128.0000 - val_mape: 136.7407
Epoch 2/20
3727/3727 ━━━━━━━━━━━━━━━━━━━━ 3s 728us/step - loss: 338549536.0000 - mape: 140.3725 - val_loss: 255361584.0000 - val_mape: 131.6407
Epoch 3/20
3727/3727 ━━━━━━━━━━━━━━━━━━━━ 3s 737us/step - loss: 214058272.0000 - mape: 119.3967 - val_loss: 81690056.0000 - val_mape: 72.8722
Epoch 4/20
3727/3727 ━━━━━━━━━━━━━━━━━━━━ 3s 731us/step - loss: 75356736.0000 - mape: 61.3909 - val_loss: 54938172.0000 - val_mape: 46.7831
Epoch 5/20
3727/3727 ━━━━━━━━━━━━━━━━━━━━ 3s 723us/step - loss: 62077096.0000 - mape: 46.8326 - val_loss: 53316252.0000 - val_mape: 43.3783
Epoch 6/20
3727/3727 ━━━━━━━━━━━━━━━━━━━━ 3s 740us/step - loss: 58398036.0000 - mape: 45.0482 - val_loss: 52654040.0000 - val_mape: 43.9704
Epoch 7/20
3727/3727 ━━━━━━━━━━━━━━━━━━━━ 3s 751us/step - loss: 57888048.0000 - mape: 45.2223 - val_loss: 52331308.0000 - val_mape: 43.5228
Epoch 8/20


In [103]:
math.sqrt(results.history["loss"][-1])

7479.027476884946

In [104]:
score = model.evaluate(x_test, y_test)

175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 512us/step - loss: 55491520.0000 - mape: 42.8772


In [105]:
print("Корень из средней квадратичной ошибки:", math.sqrt(score[0]))

Корень из средней квадратичной ошибки: 7429.192688307391


In [93]:
model.predict(np.array([0.2,	0.000,	0.666667,	0.0,	0.5,	0.6,	1.0,	0.0]).reshape(1, -1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


array([[8883.294]], dtype=float32)

In [106]:
model.save("model.h5")